<a href="https://colab.research.google.com/github/kimyeseong984/-yolo-v8/blob/main/yolo_v8_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. 기본 패키지 설치 및 YOLOv8 설치
!pip install ultralytics yt-dlp opencv-python-headless

# 2. 필요한 모듈 임포트
import cv2
import os
from ultralytics import YOLO
from pathlib import Path
from IPython.display import Video
import yt_dlp

# 3. YouTube 영상 다운로드
video_url = "https://www.youtube.com/live/S0-r1dbNQvc?si=3dsUVRJY7R6y12Cp"
output_path = "video.mp4"

ydl_opts = {
    'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
    'outtmpl': output_path
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

# 4. YOLOv8 모델 로드 (COCO 데이터셋으로 학습된 모델 사용)
model = YOLO('yolov8n.pt')  # 빠르고 가벼운 모델 (필요에 따라 yolov8s.pt, yolov8m.pt 등으로 교체 가능)

# 5. 영상에서 자동차 감지 및 분류
cap = cv2.VideoCapture(output_path)
output_dir = "car_detection_frames"
os.makedirs(output_dir, exist_ok=True)

frame_rate = 5  # 초당 5 프레임만 처리
frame_count = 0
saved_frame_count = 0

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    if frame_count % (30 // frame_rate) == 0:
        results = model(frame)
        annotated_frame = results[0].plot()

        # 감지된 객체 중 'car', 'truck', 'bus', 'motorbike'만 표시
        for r in results:
            for box in r.boxes:
                cls_id = int(box.cls[0])
                label = model.names[cls_id]
                if label in ['car', 'truck', 'bus', 'motorbike']:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(annotated_frame, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

        output_file = os.path.join(output_dir, f"frame_{saved_frame_count:04d}.jpg")
        cv2.imwrite(output_file, annotated_frame)
        saved_frame_count += 1

    frame_count += 1

cap.release()

# 6. 결과 프레임을 영상으로 변환
output_video = "detected_output.mp4"
frame_example = cv2.imread(os.path.join(output_dir, "frame_0000.jpg"))
height, width, _ = frame_example.shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, frame_rate, (width, height))

for i in range(saved_frame_count):
    frame_path = os.path.join(output_dir, f"frame_{i:04d}.jpg")
    frame = cv2.imread(frame_path)
    out.write(frame)

out.release()

# FFmpeg로 코덱을 H.264로 변환하여 호환성 향상
!apt-get -y install ffmpeg
!ffmpeg -y -i detected_output.mp4 -vcodec libx264 -acodec aac detected_output_fixed.mp4

# 영상 Colab에서 미리보기
from IPython.display import HTML

def display_local_video(video_path):
    video_file = open(video_path, "rb").read()
    video_url = "data:video/mp4;base64," + base64.b64encode(video_file).decode()
    return HTML(f"""
    <video width=640 controls>
        <source src="{video_url}" type="video/mp4">
    </video>
    """)

import base64
display_local_video("detected_output_fixed.mp4")



[youtube] Extracting URL: https://www.youtube.com/live/S0-r1dbNQvc?si=3dsUVRJY7R6y12Cp
[youtube] S0-r1dbNQvc: Downloading webpage
[youtube] S0-r1dbNQvc: Downloading tv client config
[youtube] S0-r1dbNQvc: Downloading player 73381ccc-main
[youtube] S0-r1dbNQvc: Downloading tv player API JSON
[youtube] S0-r1dbNQvc: Downloading ios player API JSON
[youtube] S0-r1dbNQvc: Downloading m3u8 information
[info] S0-r1dbNQvc: Downloading 1 format(s): 614+140
[download] video.mp4 has already been downloaded

0: 384x640 9 cars, 1 bus, 3 trucks, 1 traffic light, 9.4ms
Speed: 2.6ms preprocess, 9.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 1 bus, 3 trucks, 1 traffic light, 6.9ms
Speed: 2.9ms preprocess, 6.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 1 bus, 3 trucks, 1 traffic light, 6.9ms
Speed: 3.5ms preprocess, 6.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 4 trucks